In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, metrics, tree
import pandas as pd
from sklearn.metrics import classification_report
import json

In [2]:
with open('news_all_text.json') as file:
    file = json.load(file)

In [3]:
def set_data_frame_of_news(file, default_categories = {'استان ها', 'ورزشی', 'سیاسی', 'علمی پزشکی', 'شهروند خبرنگار'}):
    contents = []
    labels = []
    count = 0
    for data in file:
        count += 1
        if 'content' in data and 'categories' in data:
            intersection = default_categories.intersection(set(data['categories']))
            if len(intersection) != 0:
                labels.append(list(intersection)[0])
            else:
                continue
            contents.append(data['content'])
        if count > 9000:
            break
    df = pd.DataFrame({
        'contents': contents,
        'labels': labels
    })
    return df

In [4]:
df = set_data_frame_of_news(file)

In [5]:
le = preprocessing.LabelEncoder()

In [6]:
df['v_labels'] = le.fit_transform(df['labels'])
df

,contents,labels,v_labels
0,"روزنامه پیام ما با انتخاب تیتر "" جدال جاده و ج...",استان ها,0
1,مرثیه ای بر ای میراث اصفها ن در روزنامه زاینده...,استان ها,0
2,صبح امروز روزنامه خبر با تیتر « افراد گرفتار ف...,استان ها,0
3,"روزنامه خبر جنوب با تیتر "" چه کسی زیر میز می‌ز...",استان ها,0
4,منطقه سبزوار با اب هوای خشک و نیمه خشک ، در کن...,استان ها,0
...,...,...,...
4932,در حالی که رقابت‌های لیگ ملت‌های والیبال 2022 ...,ورزشی,4
4933,اخبار حاشیه‌ای ورزش ایران و جهان ، مورخ 23 خرد...,ورزشی,4
4934,اردوی تیم ملی کشتی فرنگی از روز گذشته ( یکشنبه...,ورزشی,4
4935,به گزارش خبرگزاری فرانسه اتحادیه تنیس بازان حر...,ورزشی,4


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer()
bow = cv.fit_transform(df['contents']).toarray()
bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
tf = TfidfVectorizer()
tfidf = tf.fit_transform(df['contents']).toarray()

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(bow, df['labels'], test_size=0.3, random_state=0)

In [18]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics, tree

# nb_model = GaussianNB()
nb_model = tree.DecisionTreeClassifier()


In [19]:
nb_model.fit(x_train, y_train)
y_pred = nb_model.predict(x_test)

In [20]:
from sklearn.metrics import classification_report

print('Accuracy:', metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Accuracy: 0.9770580296896086
                precision    recall  f1-score   support

      استان ها       0.97      0.94      0.96       272
         سیاسی       0.96      0.97      0.96       311
شهروند خبرنگار       0.99      1.00      0.99       288
    علمی پزشکی       0.99      0.98      0.98       306
         ورزشی       0.98      1.00      0.99       305

      accuracy                           0.98      1482
     macro avg       0.98      0.98      0.98      1482
  weighted avg       0.98      0.98      0.98      1482

